In [1]:
sc

In [5]:
!rm -rf metastore_db/*.lck

from pyspark.sql import SQLContext
sqlc = SQLContext(sc)

## PCA

In [6]:
!wget https://s3.eu-central-1.amazonaws.com/dsr-data/UScrime/UScrime2-colsLotsOfNAremoved.csv

crimes = sqlc.read.format("com.databricks.spark.csv") \
            .option("delimiter", ",") \
            .option("header", "true") \
            .option("inferSchema", "true") \
            .load("UScrime2-colsLotsOfNAremoved.csv")

--2016-09-26 20:07:02--  https://s3.eu-central-1.amazonaws.com/dsr-data/UScrime/UScrime2-colsLotsOfNAremoved.csv
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 54.231.192.53
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|54.231.192.53|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 971758 (949K) [application/octet-stream]
Saving to: ‘UScrime2-colsLotsOfNAremoved.csv.2’

100%[======================================>] 971.758     3,15MB/s   in 0,3s   

2016-09-26 20:07:03 (3,15 MB/s) - ‘UScrime2-colsLotsOfNAremoved.csv.2’ saved [971758/971758]



In [7]:
crimes = crimes.drop("OtherPerCap").drop("community")

In [9]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors, Vector

In [10]:
assembler = VectorAssembler().setInputCols(crimes.columns).setOutputCol("features")

In [11]:
featuresDF = assembler.transform(crimes).select("features")

In [12]:
from pyspark.ml.feature import PCA

pca = PCA(k=10, inputCol="features", outputCol="pca")

model = pca.fit(featuresDF)

pc = model.transform(featuresDF)

pc.toPandas()[:3]

,features,pca
0,"[0.19, 0.33, 0.02, 0.9, 0.12, 0.17, 0.34, 0.47...","[1.2138889197, 0.564567759337, -0.022284837106..."
1,"[0.0, 0.16, 0.12, 0.74, 0.45, 0.07, 0.26, 0.59...","[0.627985190195, 1.16689414866, -0.51416430664..."
2,"[0.0, 0.42, 0.49, 0.56, 0.17, 0.04, 0.39, 0.47...","[0.234349043189, 0.348070144228, 0.54876884160..."
